In [1]:
!pip install requests pandas plotly dash kaleido jupyter-dash

In [1]:
import os, time, threading, logging, requests, datetime
from collections import deque
import pandas as pd
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc, html, Output, Input, State

API_BASE       = "https://api2.arduino.cc/iot"
API_KEY        = "UyKrK5xfgO3eUEW0cOXITdpXtV2l8eBR"
API_SECRET     = "qB2LZ3vYFuNaS6c4cdXn6kOo7l90Nhnt7th9euaEVzBKsgpEhhf91Ztn7P3b6hEj"
SMART_THING_ID = "e8a146fb-8299-4fde-8672-dbe09c4dfdb9"
PROP_X_ID      = "05a482f0-7235-4b42-9837-1bf637114f0e"
PROP_Y_ID      = "36591af2-03d4-43b3-a530-0eff43eeafa1"
PROP_Z_ID      = "f24543c4-6eb5-40f9-a67a-3962be5c552c"

POLL_SECONDS   = 0.3

N_SAMPLES_PER_BATCH = 20

SAVE_DIR = "batches"
os.makedirs(SAVE_DIR, exist_ok=True)

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")

print("Notebook ready. Fill API/Thing/Property IDs above before running next cells.")

Notebook ready. Fill API/Thing/Property IDs above before running next cells.


In [2]:
def get_oauth_token():
    """Exchange API Key/Secret for OAuth token."""
    url = f"{API_BASE}/v1/clients/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": API_KEY,
        "client_secret": API_SECRET,
        "audience": API_BASE
    }
    r = requests.post(url, data=data, timeout=15)
    r.raise_for_status()
    token = r.json().get("access_token")
    if not token:
        raise RuntimeError(f"No access_token in response: {r.text}")
    return token

def read_value(token, thing_id, prop_id):
    """Read last value for a property; try metadata endpoint, then /value fallback."""
    hdr = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
    r = requests.get(f"{API_BASE}/v2/things/{thing_id}/properties/{prop_id}", headers=hdr, timeout=10)
    if r.status_code == 200:
        return r.json().get("last_value", None)
    r2 = requests.get(f"{API_BASE}/v2/things/{thing_id}/properties/{prop_id}/value", headers=hdr, timeout=10)
    if r2.status_code == 200:
        j2 = r2.json()
        return j2.get("value", j2.get("last_value", None))
    return None

def save_batch_outputs(df: pd.DataFrame):
    """Save batch CSV, stats CSV, and a PNG figure."""
    ts_tag = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_path = os.path.join(SAVE_DIR, f"batch_{ts_tag}.csv")
    png_path = os.path.join(SAVE_DIR, f"batch_{ts_tag}.png")
    stats_path = os.path.join(SAVE_DIR, f"batch_{ts_tag}_stats.csv")

    df.to_csv(csv_path, index=False)
    stats = df[["x","y","z"]].agg(["mean","std","min","max"])
    stats.to_csv(stats_path)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["t"], y=df["x"], mode="lines", name="x"))
    fig.add_trace(go.Scatter(x=df["t"], y=df["y"], mode="lines", name="y"))
    fig.add_trace(go.Scatter(x=df["t"], y=df["z"], mode="lines", name="z"))
    fig.update_layout(
        title=f"Accelerometer batch @ {ts_tag}  (N={len(df)})",
        xaxis_title="Time",
        yaxis_title="Value (m/s²)",
        legend=dict(orientation="h")
    )
    try:
        fig.write_image(png_path, scale=2, width=1000, height=400)  # requires kaleido
    except Exception as e:
        logging.warning(f"PNG save failed (install kaleido?): {e}")

    logging.info(f"Saved: {csv_path}  |  {png_path}  |  {stats_path}")

In [3]:
from collections import deque

ing_t, ing_x, ing_y, ing_z = deque(), deque(), deque(), deque()

HISTORY_MAX_SAMPLES = 600
hist_t = deque(maxlen=HISTORY_MAX_SAMPLES)
hist_x = deque(maxlen=HISTORY_MAX_SAMPLES)
hist_y = deque(maxlen=HISTORY_MAX_SAMPLES)
hist_z = deque(maxlen=HISTORY_MAX_SAMPLES)

batch_df = None
batch_version = 0

lock = threading.Lock()

def _to_float(v):
    try:
        return float(v)
    except (TypeError, ValueError):
        return None

def collector_loop():
    """Poll Cloud → append to ingress & history → when >=N, MOVE N samples to a batch; save files; bump version."""
    token = get_oauth_token()
    logging.info("Token OK. Collector started...")
    while True:
        try:
            x_raw = read_value(token, SMART_THING_ID, PROP_X_ID)
            y_raw = read_value(token, SMART_THING_ID, PROP_Y_ID)
            z_raw = read_value(token, SMART_THING_ID, PROP_Z_ID)

            fx = _to_float(x_raw)
            fy = _to_float(y_raw)
            fz = _to_float(z_raw)
            now = datetime.datetime.now()

            with lock:
                if (fx is not None) and (fy is not None) and (fz is not None):
                    ing_t.append(now); ing_x.append(fx); ing_y.append(fy); ing_z.append(fz)
                    hist_t.append(now); hist_x.append(fx); hist_y.append(fy); hist_z.append(fz)

                if len(ing_t) >= N_SAMPLES_PER_BATCH:
                    rows = []
                    for _ in range(N_SAMPLES_PER_BATCH):
                        rows.append((ing_t.popleft(), ing_x.popleft(), ing_y.popleft(), ing_z.popleft()))
                    df = pd.DataFrame(rows, columns=["t", "x", "y", "z"])
                    df["t"] = pd.to_datetime(df["t"])

                    save_batch_outputs(df)

                    global batch_df, batch_version
                    batch_df = df
                    batch_version += 1

        except requests.HTTPError as e:
            if e.response is not None and e.response.status_code == 401:
                logging.info("Token expired; refreshing...")
                token = get_oauth_token()
            else:
                logging.warning(f"HTTP error: {e}")
        except Exception as e:
            logging.warning(f"Collector error (continuing): {e}")

        time.sleep(POLL_SECONDS)

try:
    thread
    if not thread.is_alive():
        thread = threading.Thread(target=collector_loop, daemon=True)
        thread.start()
        logging.info("Collector thread restarted.")
except NameError:
    thread = threading.Thread(target=collector_loop, daemon=True)
    thread.start()
    logging.info("Collector thread started.")

2025-09-28 12:44:45,705 INFO: Collector thread started.


In [4]:
from dash import Dash, dcc, html, Output, Input, State
import plotly.graph_objects as go

app = Dash(__name__)
app.layout = html.Div([
    html.H3("Arduino Cloud → Python → Plotly (batched updates)"),
    html.Div([
        html.Label("Current batch version:"),
        html.Span(id="ver-label", style={"fontWeight": "bold", "marginLeft": "8px"})
    ]),
    dcc.Graph(id="graph-combined"),
    dcc.Graph(id="graph-x"),
    dcc.Graph(id="graph-y"),
    dcc.Graph(id="graph-z"),
    dcc.Interval(id="tick", interval=1500, n_intervals=0),
    dcc.Store(id="ver-store", data={"ver": 0}),
])

@app.callback(
    Output("graph-combined", "figure"),
    Output("graph-x", "figure"),
    Output("graph-y", "figure"),
    Output("graph-z", "figure"),
    Output("ver-store", "data"),
    Output("ver-label", "children"),
    Input("tick", "n_intervals"),
    State("ver-store", "data"),
    prevent_initial_call=False
)
def refresh(_n, ver_data):
    with lock:
        if len(hist_t) == 0:
            empty = go.Figure()
            return empty, empty, empty, empty, ver_data, f"{ver_data.get('ver', 0)} (waiting...)"
        df = pd.DataFrame({
            "t": list(hist_t),
            "x": list(hist_x),
            "y": list(hist_y),
            "z": list(hist_z)
        })
        current_ver = batch_version

    figc = go.Figure()
    figc.add_trace(go.Scatter(x=df["t"], y=df["x"], mode="lines", name="x"))
    figc.add_trace(go.Scatter(x=df["t"], y=df["y"], mode="lines", name="y"))
    figc.add_trace(go.Scatter(x=df["t"], y=df["z"], mode="lines", name="z"))
    figc.update_layout(title=f"Combined (batch {current_ver})", xaxis_title="Time", yaxis_title="m/s²")

    figx = go.Figure([go.Scatter(x=df["t"], y=df["x"], mode="lines", name="x")])
    figx.update_layout(title="X axis", xaxis_title="Time", yaxis_title="m/s²")

    figy = go.Figure([go.Scatter(x=df["t"], y=df["y"], mode="lines", name="y")])
    figy.update_layout(title="Y axis", xaxis_title="Time", yaxis_title="m/s²")

    figz = go.Figure([go.Scatter(x=df["t"], y=df["z"], mode="lines", name="z")])
    figz.update_layout(title="Z axis", xaxis_title="Time", yaxis_title="m/s²")

    return figc, figx, figy, figz, {"ver": current_ver}, str(current_ver)

In [5]:
app.run(jupyter_mode="external", debug=False, port=8050)

Dash app running on http://127.0.0.1:8050/
